In [1]:
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.utils import getResolvedOptions
from awsglue.transforms import Map, DropFields, ApplyMapping

sc = SparkContext()
context = GlueContext(sc)

In [2]:
path = "s3://network.cubo.datalake/airtable/raw/prod/"
path = path + "2023-09-04-02-34-19/data/"
path = path + "Conexões - Base Eventos/Conexões/Grid Eventos/"
print(path)
dataframeBaseEventos = context.create_dynamic_frame.from_options(
                connection_type='s3',
                connection_options={
                    'paths': [path],
                    'recurse': True
                },
                format='json'
            )
dataframeBaseEventos.count()

s3://network.cubo.datalake/airtable/raw/prod/2023-09-04-02-34-19/data/Conexões - Base Eventos/Conexões/Grid Eventos/


246

In [3]:
dataframeBaseEventos.printSchema()

root
|-- id: string
|-- createdTime: string
|-- fields: struct
|    |-- Createdat: string
|    |-- Name: string
|    |-- Type: string
|    |-- Date: string
|    |-- Event: string
|    |-- Status: string



In [4]:
mapping = []
for item in dataframeBaseEventos.unnest().toDF().dtypes:
    if item[0].split('.')[0] == "fields":
        if len(item[0].split(".")) == 2:
            mapping.append((item[0], item[0].split('.')[1]))
    else:
        mapping.append((item[0], item[0].split('.')[0]))  

In [6]:
newDf = dataframeBaseEventos.apply_mapping(mapping).resolveChoice([("Createdat", "cast:string")]) \
        .drop_fields(["Date"])

In [7]:
newDf.printSchema()

root
|-- id: string
|-- createdTime: string
|-- Createdat: string
|-- Name: string
|-- Type: string
|-- Event: string
|-- Status: string

